In [1]:
import gurobipy
from datetime import datetime
import pandas as pd

In [36]:
# Definir conjuntos T y K
T = range(1, 25)  # Conjunto T: {1, 2, ..., 24}
K = range(1, 366)  # Conjunto K: {1, 2, ..., 365}
# K = [1]

#Parámetros
F = 40 #Capacidad de carga
alpha = 20 #Flujo de carga

In [37]:
#INCORPORAR EL CONSUMO HORARIO "D" (DATOS GENERADOS AL AZAR)

# Lee el archivo Excel
fd = pd.read_excel('consumo_electrico.xlsx', header=None)

# Elimina la primera fila que contiene los nombres de las columnas
fd = fd.drop(0)

fd = fd / 10

# Define los nombres de las columnas manualmente
kolumn_names = ['V'+str(i).zfill(2) for i in range(1, 25)]

# Asigna los nombres de las columnas al DataFrame
fd.columns = kolumn_names

# Crea la variable D[t, k] que almacene el valor para la hora t del día k
D = {}

# Recorre cada fila del DataFrame y actualiza el valor para cada hora t del día k
for index, row in fd.iterrows():
    k = index   # Ajusta el índice de fila para comenzar desde el día 2
    for t in range(1, 25):
        D[(t, k)] = row['V'+str(t).zfill(2)]
#print(D)

In [38]:
#INCORPORAR EL COSTO MARGINAL

# Lee el archivo CSV sin encabezado y con punto y coma como delimitador
df = pd.read_csv('cerro_navia.csv', header=None, delimiter=';')

# Define los nombres de las columnas manualmente
column_names = ['fecha', 'dia', 'hora', 'costo_en_dolares', 'nombre']

# Asigna los nombres de las columnas al DataFrame
df.columns = column_names

# Elimina la primera fila que contiene la palabra "fecha"
df = df.drop(0)

# Reemplaza las comas por puntos en la columna 'costo_en_dolares'
df['costo_en_dolares'] = df['costo_en_dolares'].str.replace(',', '.')

# Convierte la columna 'costo_en_dolares' a tipo numérico
df['costo_en_dolares'] = pd.to_numeric(df['costo_en_dolares'], errors='coerce')

# Divide la columna 'fecha' en 'dia', 'mes' y 'ano'
df[['dia', 'mes', 'ano']] = df['fecha'].str.split('-', expand=True)

# Elimina la columna 'fecha' ya que ahora está dividida en 'dia', 'mes' y 'ano'
df.drop(columns=['fecha'], inplace=True)

# Convierte las columnas 'dia', 'mes' y 'ano' a tipo numérico
df['dia'] = pd.to_numeric(df['dia'], errors='coerce')
df['mes'] = pd.to_numeric(df['mes'], errors='coerce')
df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

# Crea la variable P[t, k] que almacene el costo para el día k a la hora t
P = {}
def fecha(ano, mes, dia):
  fechaa = datetime((row["ano"]), (row["mes"]), (row['dia']))
  return fechaa.toordinal() - 738520
# Recorre cada fila del DataFrame y actualiza el costo para cada día k a la hora t
for index, row in df.iterrows():
    t = int(row['hora'])
    # Verifica si los valores de año, mes y día son válidos antes de crear el objeto datetime
    if pd.notnull(row['ano']) and pd.notnull(row['mes']) and pd.notnull(row['dia']):
        #fecha = datetime((row["ano"]), (row["mes"]), (row['dia']))
        k = int(fecha((row["ano"]), (row["mes"]), (row['dia'])))
        costo = row['costo_en_dolares']
        if (t, k) not in P:
            P[(t, k)] = costo

#print(P)

In [39]:
#GUROBI
import gurobipy as gp
from gurobipy import GRB

# Crear el modelo
problema = gp.Model("problema")

# Definir variables de decisión
x = {(t, k): problema.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f"x[{t},{k}]") for t in T for k in K}
z = {(t, k): problema.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"z[{t},{k}]") for t in T for k in K}

# Definir la función objetivo
problema.setObjective(gp.quicksum(((1 - x[(t, k)]) * D[(t, k)] * P[(t, k)] + z[(t, k)] * P[(t, k)]) for k in K for t in T), GRB.MINIMIZE)

# RESTRICCIONES

## RESTRICCION DE INVENTARIO
# Definir la variable auxiliar I
I = {(t, k): problema.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"I[{t},{k}]") for t in T for k in K}

# Condiciones de borde para I
problema.addConstr(I[(1, 1)] == 0, name="borde1")
for k in K:
    for t in T:
        problema.addConstr(I[(t, k)] <= F, name=f"borde2_{t}_{k}")
        problema.addConstr(I[(t, k)] >= 0, name=f"borde3_{t}_{k}")

# Definición de I
for k in K:
    for t in range(2, 25):
        problema.addConstr(I[(t, k)] == I[(t-1, k)] + z[(t-1, k)] - x[(t-1, k)] * D[(t-1, k)], name=f"carga_{t}_{k}")
    if k >= 2:
        problema.addConstr(I[(1, k)] == I[(24, k-1)] + z[(24, k-1)] - x[(24, k-1)] * D[(24, k-1)], name=f"borde4_{k}")
for k in K:
    for t in T:
        M = 10**6
        problema.addConstr(M * I[(t, k)] >= x[(t, k)], name=f"restriccion_{t}_{k}")

## RESTRICCIÓN DE FLUJO:
for t in T:
    for k in K:
        problema.addConstr(x[(t, k)] * D[(t, k)] + z[(t, k)]*12 <= alpha, name=f"flujo_{t}_{k}")

# Resolver el problema
problema.optimize()

# Imprimir resultados
print("Estado:", problema.status)
if problema.status == GRB.OPTIMAL:
    for t in T:
        print(f"Valor de I({t}, {1}):", I[(t, 1)].x)
        print(f"Valor de xI({t}, {1}):", x[(t, 1)].x)
        print(f"Valor de z({t}, {1}):", z[(t, 1)].x)
print("Valor óptimo:", problema.objVal)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43800 rows, 26280 columns and 87597 nonzeros
Model fingerprint: 0xb78ae2e9
Coefficient statistics:
  Matrix range     [9e-04, 1e+06]
  Objective range  [1e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 4e+01]
Presolve removed 17525 rows and 4 columns
Presolve time: 0.18s
Presolved: 26275 rows, 26276 columns, 70066 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.06s

Barrier statistics:
 AA' NZ     : 4.379e+04
 Factor NZ  : 3.194e+05 (roughly 24 MB of memory)
 Factor Ops : 5.325e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual    

In [24]:
resultados(1)


Hora 1
 x: %Energia sacada de la bateria para suministrar = 0.0
 z: Energia comprada para almacenar en la bateria en t,k = 1e-06
 I: Energia almacenada en la bateria = 0.0


Hora 2
 x: %Energia sacada de la bateria para suministrar = 1.0
 z: Energia comprada para almacenar en la bateria en t,k = 0.0198918530540876
 I: Energia almacenada en la bateria = 1e-06


Hora 3
 x: %Energia sacada de la bateria para suministrar = 0.0
 z: Energia comprada para almacenar en la bateria en t,k = 0.0
 I: Energia almacenada en la bateria = 0.0


Hora 4
 x: %Energia sacada de la bateria para suministrar = 0.0
 z: Energia comprada para almacenar en la bateria en t,k = 1.48922119671756
 I: Energia almacenada en la bateria = 0.0


Hora 5
 x: %Energia sacada de la bateria para suministrar = 1.0
 z: Energia comprada para almacenar en la bateria en t,k = 0.0
 I: Energia almacenada en la bateria = 1.48922119671756


Hora 6
 x: %Energia sacada de la bateria para suministrar = 0.0
 z: Energia comprada para alma

In [40]:
gastofeo = sum([D[t,k]*P[t,k] for t in T for k in K])
gastofeo

1535583.9627957537

In [41]:
problema.objVal

860231.3901596909

In [42]:
round(gastofeo - problema.objVal,2)

675352.57

In [47]:
ahorro = (1 - problema.objVal/gastofeo) * 100
ahorro

43.98017881135495

In [7]:

def resultados(dia: int):

    for t in range(1,25):
        print()

        print(f'Hora {t}')
        print(f' x: %Energia sacada de la bateria para suministrar = {x[t, dia].x}')
        print(f' z: Energia comprada para almacenar en la bateria en t,k = {z[t, dia].x}')
        print(f' I: Energia almacenada en la bateria = {I[t, dia].x}')

        print()

In [28]:
c = 0
for t in range(1,365):
    c += consumo.iloc[t].sum()/24
print(c)
print(c/365)

8198.093827212446
22.460531033458757


In [ ]:
# ahorro anual
# ahorro diario
# ahorro en 1 hora